In [3]:
import os
import glob
import cv2
import numpy as np


In [9]:
base_dir ="./rec1_processed"
img_dir = os.path.join("x","*.png")
#print(img_dir) 
lab_dir = os.path.join("y", "*.png")

num_images =100

imgs = glob.glob(img_dir)
labs = glob.glob(lab_dir)
imgs.sort()
labs.sort()

imgs=imgs[0:num_images]
labs = labs[0:num_images]
training_paths = [imgs, labs]
training_paths = np.asarray(training_paths)
print(training_paths.shape)
training_paths = training_paths.T
print(training_paths.shape)


print(training_paths[0][0])
print(training_paths[0][1])

#print(labs[0])
#print(imgs[0])

training_paths = [imgs, labs]


def get_image_slices(img, label=False):
    
    dim = 2560
    imgs = np.empty([4, 640, 640, 3])
    imgs[0] = img[0:int(dim/4), 0:int(dim/4), :]
    imgs[1] = img[int(dim/4):int(dim/2), int(dim/4):int(dim/2), :]
    imgs[2] = img[int(dim/2):int(3*dim/4), int(dim/2):int(3*dim/4), :]
    imgs[3] = img[int(3*dim/4):int(dim), int(3*dim/4):int(dim), :]

    return np.asarray(imgs)

def get_training_set(img):
    raw_img = cv2.imread(img[0])
    raw_label = cv2.imread(img[1])
  
    imgs = get_image_slices(raw_img)
    labels = get_image_slices(raw_label, True)
  
    return imgs, labels
training_paths = np.asarray(training_paths)


(2, 100)
(100, 2)
x/frame_0001.png
y/frame_0001.png


In [10]:
img = cv2.imread("x/frame_0001.png")
arr = ["x/frame_0001.png", "y/frame_0001.png"]
#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img, label = get_training_set(arr)
print(img.shape)

(4, 640, 640, 3)


In [11]:
import multiprocessing as mp
import time


slice_num = 10

pool = mp.Pool(mp.cpu_count())

print("Training paths %4d"%(len(training_paths)))

slices = int(len(training_paths.shape[0])/slice_num)

print(slices)
start_time = time.time()
res = np.array([])
imgs_ready = np.empty([0, 640, 640]) 
labels_ready = np.empty([0, 640, 640, 2]) 

for i in range(1):
    slice_start = time.time()
    print("%d. slice has been started being loaded."%(i+1))
    
    cur_slice = training_paths[int(i*slices):int((i+1)*slices)]
    
    print("The current slize's shape is: ")
    print(np.asarray(cur_slice).shape)
    x = [pool.map(get_training_set, [img_pair for img_pair in cur_slice])]
    
    x = np.asarray(x)
    print("The loaded images' shape is:")
    print(x.shape)
    x = x.reshape(2, 4*slices, 640, 640, 3)
    print(x.shape)
    slice_end = time.time()
    took = (slice_end-slice_start)
    print("{0:1d}. slice has finished in {1:10.5f} seconds.".format((i+1), took))
    im_temp = x[0, :, :, :, 0]
    print(im_temp.shape)
    lab_temp = x[1, :, :, :, 0:2]
    print(lab_temp.shape)
    imgs_ready = np.vstack((imgs_ready, im_temp))
    labels_ready = np.vstack((labels_ready, lab_temp))
    
end_time = time.time()
full_took = end_time - start_time
print("Total time: {0:10.5f} seconds".format(full_took))
pool.close()

Training paths    2


TypeError: object of type 'int' has no len()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras import losses
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
plt.imshow(labels_ready[0, :, :, 0], cmap= "gray")

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
input_img = Input((640, 640, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True)
model.load_weights("./ckpts/tb4/best_1.hdf5")

In [ ]:

selected_loss = "binary_crossentropy"
model.compile("adam", loss=selected_loss, metrics=["accuracy", losses.mean_squared_error, losses.logcosh, losses.kullback_leibler_divergence,  losses.binary_crossentropy, losses.mean_squared_logarithmic_error])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("./ckpts/un/50_10_10ep_best.hdf5", save_best=1, monitor="val_loss", mode="auto")
tboard = tf.keras.callbacks.TensorBoard("./ckpts/un/50", histogram_freq=1, batch_size=10, write_graph = True, write_grads = True, write_images = True, update_freq = "batch")
callbacks = [checkpoint, tboard]

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2,
                               rotation_range=15,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.01,
                               zoom_range=[0.5, 1.25],
                               horizontal_flip=True,
                               vertical_flip=True,
                               fill_mode='reflect',
                               brightness_range=[0.5, 1.5])#rotation_range=360, fill_mode = "reflect", horizontal_flip=1, vertical_flip=1,)
#datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_std_normalization= 1, rotation_range=360, fill_mode = "reflect", horizontal_flip=1, vertical_flip=1, validation_split=0.2)
#you have to use datagen.fit if u want to use featurewise_std_normalization so that the generator learns about the features

In [ ]:
history = model.fit_generator(datagen.flow(x=imgs_ready.reshape(1600,640,640,1), y=labels_ready.reshape(1600,640,640,1), batch_size=10, shuffle=True), steps_per_epoch = 10, epochs = 50, verbose = True, callbacks = callbacks, use_multiprocessing = False)

In [ ]:
preds = model.predict(loaded_imgs, batch_size = 10)

In [ ]:
print(preds.shape)

In [ ]:
#fig = plt.Figure(figsize=(15,15))
#grids = ImageGrid(fig, 111, nrows_ncols=(2,2))

fig = plt.figure(figsize=(20,100))
#nrows, ncols, and index 
grids = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(10, 2),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
z = 0
for i in range(10):  
    test_im=preds[i]
    original = loaded_imgs[i].reshape((640,640))
    test_im = test_im*255
    test_im = test_im.reshape((640,640))
    test_im = test_im.astype(int)
    z = 2*i
    #print(test_im)  
    grid1 = grids[z]
    
    grid2 = grids[z+1]
    
    grid1.imshow(original, cmap="gray")
    grid2.imshow(test_im, cmap = "gray")

In [ ]:
fig.savefig("promising_results.pdf")

In [ ]:
print("hello")